In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import helper, op, script, tx

### Exercise 1




#### Make [this test](/edit/session5/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [ ]:
# Exercise 1

reload(tx)
run(tx.TxTest('test_verify_p2pkh'))

In [ ]:
# Transaction Construction Example
from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut
# Step 1
tx_ins = []
prev_tx = bytes.fromhex('95ae966a54493321316799d5c4ba7f48faadd653b7eb725ea2c5ea7d38f65cff')
prev_index = 0
tx_ins.append(TxIn(prev_tx, prev_index))
# Step 2
tx_outs = []
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.37*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.1*100000000),
    script_pubkey=p2pkh_script(h160),
))
tx_obj = Tx(1, tx_ins, tx_outs, 0, network="signet")
# Step 3
z = tx_obj.sig_hash(0)
pk = PrivateKey(secret=8675309)
der = pk.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = pk.point.sec()
tx_obj.tx_ins[0].script_sig = Script([sig, sec])
print(tx_obj.serialize().hex())

### Exercise 2




#### Make [this test](/edit/session5/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [ ]:
# Exercise 2

reload(tx)
run(tx.TxTest('test_sign_input'))

### Exercise 3
You have been sent 100,000 Sats on the Signet network. Send 40,000 Sats to this address: `mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv` and send the rest back to yourself.

#### Send your transaction here: https://mempool.space/signet/tx/push


In [ ]:
# Exercise 3

from tx import Tx, TxIn, TxOut
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script
prev_tx = bytes.fromhex('<transaction id here>')  # CHANGE
prev_index = -1  # CHANGE
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
target_amount = 40000
fee = 300
passphrase = b'<your passphrase here>'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
change_address = private_key.point.address(network="signet")
# initialize inputs

# create a new tx input with prev_tx, prev_index

# initialize outputs

# decode the hash160 from the target address

# convert hash160 to p2pkh script

# create a new tx output for target with amount and script_pubkey

# decode the hash160 from the change address

# convert hash160 to p2pkh script

# get the value for the transaction input (remember network="signet")

# calculate change_amount based on previous amount, target_amount & fee

# create a new tx output for change with amount and script_pubkey

# create the transaction (name it tx_obj and set network="signet")

# now sign the 0th input with the private_key using sign_input

# SANITY CHECK: change address corresponds to private key
if private_key.point.address(network="signet") != change_address:
    raise RuntimeError('Private Key does not correspond to Change Address, check priv_key and change_address')
# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_ins[0].script_pubkey(network="signet").commands[2] != decode_base58(change_address):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')
# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 100000 or tx_obj.fee() <= 0:
    raise RuntimeError(f'Check that the change amount is reasonable. Fee is {tx_obj.fee()}')
# serialize and hex()


### Exercise 4
#### Bonus Question. Only attempt if you've finished Exercise 3 and have time to try it.

Get some signet coins from a faucet and spend both outputs (one from your change address and one from the signet faucet) to

`mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv`

#### You can get some free signet coins at: https://signet.bc-2.jp/


In [ ]:
# Exercise 4

# Bonus
from tx import Tx, TxIn, TxOut
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script
prev_tx_1 = bytes.fromhex('<tx id from last exercise>')  # CHANGE
prev_index_1 = -1  # CHANGE
prev_tx_2 = bytes.fromhex('<tx id from faucet>')  # CHANGE
prev_index_2 = -1  # CHANGE
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
fee = 400
passphrase = b'<your passphrase here>'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
# initialize inputs

# create the first tx input with prev_tx_1, prev_index_1

# create the second tx input with prev_tx_2, prev_index_2

# initialize outputs

# decode the hash160 from the target address

# convert hash160 to p2pkh script

# calculate target amount by adding the input values and subtracting the fee

# create a single tx output for target with amount and script_pubkey

# create the transaction

# sign both inputs with the private key using sign_input


# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_ins[0].script_pubkey(network="signet").commands[2] != decode_base58(private_key.point.address(network="signet")):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')
# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 100000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {tx_obj.fee()}')
# serialize and hex()


In [ ]:
# op_checkmultisig
def op_checkmultisig(stack, z):
    if len(stack) < 1:
        return False
    n = decode_num(stack.pop())
    if len(stack) < n + 1:
        return False
    sec_pubkeys = []
    for _ in range(n):
        sec_pubkeys.append(stack.pop())
    m = decode_num(stack.pop())
    if len(stack) < m + 1:
        return False
    der_signatures = []
    for _ in range(m):
        # signature is assumed to be using SIGHASH_ALL
        der_signatures.append(stack.pop()[:-1])
    # OP_CHECKMULTISIG bug
    stack.pop()
    try:
        raise NotImplementedError
    except (ValueError, SyntaxError):
        return False
    return True

### Exercise 5




#### Make [this test](/edit/session5/op.py) pass: `op.py:OpTest:test_op_checkmultisig`

In [ ]:
# Exercise 5

reload(op)
reload(script)
run(op.OpTest('test_op_checkmultisig'))

### Exercise 6
Find the hash160 of the RedeemScript
```
5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```


In [ ]:
# Exercise 6

from helper import hash160
hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'
# bytes.fromhex script

# hash160 result

# hex() to display


In [ ]:
# P2SH address construction example
from helper import encode_base58_checksum
print(encode_base58_checksum(b'\x05'+bytes.fromhex('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))

### Exercise 7




#### Make [this test](/edit/session5/helper.py) pass: `helper.py:HelperTest:test_p2pkh_address`

In [ ]:
# Exercise 7

reload(helper)
run(helper.HelperTest('test_p2pkh_address'))

### Exercise 8




#### Make [this test](/edit/session5/helper.py) pass: `helper.py:HelperTest:test_p2sh_address`

In [ ]:
# Exercise 8

reload(helper)
run(helper.HelperTest('test_p2sh_address'))

### Exercise 9




#### Make [this test](/edit/session5/script.py) pass: `script.py:ScriptTest:test_address`

In [ ]:
# Exercise 9

reload(script)
run(script.ScriptTest('test_address'))

In [ ]:
# z for p2sh example
from helper import hash256
h256 = hash256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(h256, 'big')
print(hex(z))

In [ ]:
# p2sh verification example
from ecc import S256Point, Signature
from helper import hash256
h256 = hash256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(h256, 'big')
point = S256Point.parse(bytes.fromhex('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
sig = Signature.parse(bytes.fromhex('3045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937'))
print(point.verify(z, sig))

### Exercise 10
Validate the second signature of the first input

```
0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000
```

The sec pubkey of the second signature is:
```
03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71
```

The der signature of the second signature is:
```
3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022
```

The redeemScript is:
```
475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```


In [ ]:
# Exercise 10

from io import BytesIO
from ecc import S256Point, Signature
from helper import int_to_little_endian, SIGHASH_ALL
from script import Script
from tx import Tx
hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71'
hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022'
hex_redeem_script = '475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'
sec = bytes.fromhex(hex_sec)
der = bytes.fromhex(hex_der)
redeem_script_stream = BytesIO(bytes.fromhex(hex_redeem_script))
redeem_script = Script.parse(redeem_script_stream)
hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000'
stream = BytesIO(bytes.fromhex(hex_tx))
# parse the S256Point and Signature


# parse the Tx

# change the first input's ScriptSig to RedeemScript

# get the serialization

# add the sighash (4 bytes, little-endian of SIGHASH_ALL)

# hash256 the result

# your z is the hash256 as a big-endian number: use int.from_bytes(x, 'big')

# now verify the signature using point.verify
